In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# ##### 各関数での実行時間を保存するためのCSVの空データを作成するための処理 #####
# Processes = []
# for i in range(1,16):
#     Processes.append(i*i)
# for j in range(9):
#     Processes.append(2**j)
# Processes = list(set(Processes))
# Processes.sort()
# index = classes
# EmptyDF = pd.DataFrame(index=index, columns=Processes)
# for benchmark in benchmarks:
#     EmptyDF.to_csv(f"./csv_files/ExecTime@{benchmark}.csv")

##### 各ベンチマークの実行時間を記録する処理
# # Benchmarks = []
# # for benchmark in benchmarks:
# #     Benchmarks.append(benchmark.upper())
# # print(Benchmarks)

# Processes = [8, 16, 32, 64, 128, 256]
# for process in Processes:
#     TimeDataList = return_TimeDataList(process)
#     for DictData in TimeDataList:
#         benchmark = DictData.benchmark
#         process = DictData.process
#         BenchmarkClass = DictData.BenchmarkClass
#         time = DictData.time
#         FillCSV(benchmark=benchmark, process=process, BenchmarkClass=BenchmarkClass, time=time)

# # print(TimeDataList[0].return_AllData())


In [ ]:
# 論文で必要なグラフを生成する1
print("↓プロセス数を固定したもの")
GenGraphAveragePerProfileNum(benchmarks=benchmarks, Fixed="Process", Fix=64, Predict="D", EnableTitle=True)
print("↓ベンチマーククラスを固定したもの")
GenGraphAveragePerProfileNum(benchmarks=benchmarks, Fixed="Class", Fix="B", Predict=256, EnableTitle=True)

In [ ]:
# 論文で必要なグラフを生成する2
for benchmark in benchmarks:
    print(benchmark)
    print("↓プロセス数を固定したもの")
    GenGraphTotalTimePerProfileNum(benchmark=benchmark, Fixed="Process", Fix=64, Predict="D", EnableTitle=True)
    print("↓ベンチマーククラスを固定したもの")
    GenGraphTotalTimePerProfileNum(benchmark=benchmark, Fixed="Class", Fix="B", Predict=256, EnableTitle=True)
    

In [ ]:
# 論文で必要なグラフを生成する3
benchmarks01 = ['cg', 'ep', 'mg']
benchmarks02 = ['ft', 'lu']
benchmarks03 = ["bt", "sp"]
benchmarks04 = ['is']
print(f"benchmarks01 = {benchmarks01}, benchmarks02 = {benchmarks02}, benchmarks03 = {benchmarks03}")
print(f"↓ベンチマーククラスを固定したもの@{benchmarks01}")
GenGraphAveragePerProfileNum(benchmarks=benchmarks01, Fixed="Class", Fix="B", Predict=256, EnableTitle=True)
print(f"↓ベンチマーククラスを固定したもの@{benchmarks02}")
GenGraphAveragePerProfileNum(benchmarks=benchmarks02, Fixed="Class", Fix="B", Predict=256, EnableTitle=True)
print(f"↓ベンチマーククラスを固定したもの@{benchmarks03}")
GenGraphAveragePerProfileNum(benchmarks=benchmarks03, Fixed="Class", Fix="B", Predict=256, EnableTitle=True)
print(f"↓ベンチマーククラスを固定したもの@{benchmarks04}")
GenGraphAveragePerProfileNum(benchmarks=benchmarks04, Fixed="Class", Fix="B", Predict=256, EnableTitle=True)